Pretty much this is just a file which steals and try to understand Mo's code for integration. To understand the code, see Mo's notebooks.

In [14]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import time

from catboost import CatBoostRegressor

import re

from statsmodels.stats.outliers_influence import variance_inflation_factor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler

import pickle

pd.options.display.max_columns = 250;

In [15]:
housing_coords = pd.read_csv('../data/ames_housing_price_data_v6.csv', index_col = 0);
housing_coords.shape

# housing_coords["Address"] = housing_coords["Coords4"].apply(lambda x: x[-21])

(2471, 38)

In [16]:
housing_coords.head()

,SalePrice,GrLivArea,LotArea,OverallQual,BSMT_LowQual,house_age_years,GarageCars,MasVnrType,FullBath,HalfBath,BsmtExposure_ord,SaleTypeNew,Neighborhood,BldgType,PorchSF,BSMT_HighQual,Fireplaces,Pool,BedroomAbvGr,ExterQual,OverallCond,KitchenQual,water_tower,graveyard,police,optician,slipway,bar,cinema,supermarket,hotel,stop,farmyard,christian_catholic,jewish,muslim,garden_centre,christian_lutheran
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
909176150,126000,856,7890,0.428571,856.0,71.210959,2.0,None,1,0,1,0,SWISU,1Fam,166,0.0,1,0,2,0.333333,0.500000,0.333333,2,1,1,1,0,11,0,4,0,0,0,1,0,0,0,3
905476230,139500,1049,4235,0.285714,104.0,25.104110,1.0,Brick Face,2,0,2,0,Edwards,TwnhsE,105,945.0,0,0,2,0.666667,0.333333,0.666667,1,1,1,1,0,11,0,4,0,0,0,1,0,1,0,2
535377150,114000,1039,8146,0.142857,405.0,109.402740,1.0,None,1,0,1,0,OldTown,1Fam,279,0.0,0,0,2,0.666667,0.833333,0.333333,0,1,1,1,0,4,1,1,0,0,1,0,0,0,0,3
534177230,227000,1665,8400,0.714286,167.0,8.838356,2.0,None,2,1,1,0,NWAmes,1Fam,45,643.0,0,0,3,0.666667,0.500000,0.666667,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,3
908128060,198500,1922,7301,0.571429,0.0,6.501370,2.0,Brick Face,3,0,0,0,Edwards,1Fam,177,0.0,1,0,4,0.666667,0.333333,0.666667,1,0,0,1,0,2,0,4,3,1,0,0,0,0,0,1


In [4]:
for col in column_title_dict:
#    print(column_title_dict[col]["Category"]);
    if column_title_dict[col]["Category"] == "Continuous":
        print(housing_coords[col].min(), housing_coords[col].max(), col)

334 4316 GrLivArea
1300 215245 LotArea
0.0 2336.0 BSMT_LowQual
0.0 2306.0 BSMT_HighQual
0.1616438356164383 137.0082191780822 house_age_years
0 1207 PorchSF


In [17]:
temp = housing_coords["LotArea"]
temp2 = housing_coords
temp2.drop("LotArea", axis = 1, inplace = True)

In [20]:
temp2["LotArea"] = temp;

In [21]:
temp2.head()

,SalePrice,GrLivArea,OverallQual,BSMT_LowQual,house_age_years,GarageCars,MasVnrType,FullBath,HalfBath,BsmtExposure_ord,SaleTypeNew,Neighborhood,BldgType,PorchSF,BSMT_HighQual,Fireplaces,Pool,BedroomAbvGr,ExterQual,OverallCond,KitchenQual,water_tower,graveyard,police,optician,slipway,bar,cinema,supermarket,hotel,stop,farmyard,christian_catholic,jewish,muslim,garden_centre,christian_lutheran,LotArea
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
909176150,126000,856,0.428571,856.0,71.210959,2.0,None,1,0,1,0,SWISU,1Fam,166,0.0,1,0,2,0.333333,0.500000,0.333333,2,1,1,1,0,11,0,4,0,0,0,1,0,0,0,3,7890
905476230,139500,1049,0.285714,104.0,25.104110,1.0,Brick Face,2,0,2,0,Edwards,TwnhsE,105,945.0,0,0,2,0.666667,0.333333,0.666667,1,1,1,1,0,11,0,4,0,0,0,1,0,1,0,2,4235
535377150,114000,1039,0.142857,405.0,109.402740,1.0,None,1,0,1,0,OldTown,1Fam,279,0.0,0,0,2,0.666667,0.833333,0.333333,0,1,1,1,0,4,1,1,0,0,1,0,0,0,0,3,8146
534177230,227000,1665,0.714286,167.0,8.838356,2.0,None,2,1,1,0,NWAmes,1Fam,45,643.0,0,0,3,0.666667,0.500000,0.666667,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,3,8400
908128060,198500,1922,0.571429,0.0,6.501370,2.0,Brick Face,3,0,0,0,Edwards,1Fam,177,0.0,1,0,4,0.666667,0.333333,0.666667,1,0,0,1,0,2,0,4,3,1,0,0,0,0,0,1,7301


In [4]:
column_title_dict = {
    ### from original dataset
    'GrLivArea' : {"Category": "Continuous", "Description": 'Above-ground living area', "Actionable":True, "Range": (300,4400,10)},
    'LotArea' : {"Category": "Continuous", "Description": 'Lot area', "Actionable":False},
    'OverallQual' : {"Category": "Ordinal", "Description": 'Overall quality', "Actionable":True},
    'BSMT_LowQual' : {"Category": "Continuous", "Description": 'Low-quality basement area', "Actionable":True, "Range": (0,2400,10)},
    'BSMT_HighQual' : {"Category": "Continuous", "Description": 'High-quality basement area', "Actionable":True, "Range": (0,2400,10)},
    'house_age_years' : {"Category": "Continuous", "Description": 'House age in years', "Actionable":False},
    'GarageCars' : {"Category": "Discrete", "Description": 'Number of cars held by garage', "Actionable":True},
    'MasVnrType' : {"Category": "Nominal", "Description": 'Masonry veneer type', "Actionable":True},
    'FullBath' : {"Category": "Discrete", "Description":'Number of full bathrooms', "Actionable":True},
    'HalfBath' : {"Category": "Discrete", "Description":'Number of half-bathrooms', "Actionable":True},
    'BsmtExposure_ord' : {"Category": "Ordinal", "Description":'Basement exposure', "Actionable":True},
    'Neighborhood' : {"Category": "Nominal", "Description":'Neighborhood', "Actionable":False},
    'BldgType' : {"Category": "Nominal", "Description":'Building type', "Actionable":False},
    'PorchSF' : {"Category": "Continuous", "Description":'Porch area in sq ft', "Actionable":True, "Range": (0,1250,5)},
    'ExterQual' : {"Category": "Ordinal", "Description":'Exterior quality score', "Actionable":True},
    'OverallCond' : {"Category": "Ordinal", "Description":'Overall condition score', "Actionable":True},
    'KitchenQual' : {"Category": "Ordinal", "Description":'Kitchen quality score', "Actionable":True},
    'Fireplaces' : {"Category": "Discrete", "Description":'Number of fireplaces', "Actionable":True},
    'Pool' : {"Category": "Discrete", "Description":'Pool', "Actionable":True},
    'BedroomAbvGr' : {"Category": "Discrete", "Description":'Number of bedrooms', "Actionable":True},
    # 'ext_Asbestos_Shingles' : {"Category": "Nominal", "Description":'Asbestos used in walls'},
    
    ### location features
    'graveyard' : {"Category": "Discrete", "Description":'Number graveyards within 1 mile', "Actionable":False},
    'police' : {"Category": "Discrete", "Description":'Number of police stations within 1 mile', "Actionable":False},
    'optician' : {"Category": "Discrete", "Description":'Number of opticians within 1 mile', "Actionable":False},
    'stop' : {"Category": "Discrete", "Description":'Number of stop signs within 1 mile', "Actionable":False},
    'slipway' : {"Category": "Discrete", "Description":'Number of slipways within 1 mile', "Actionable":False},
    'bar' : {"Category": "Discrete", "Description":'Number of bars within 1 mile', "Actionable":False},
    'cinema' : {"Category": "Discrete", "Description":'Number of cinemas within 1 mile', "Actionable":False},
    'supermarket' : {"Category": "Discrete", "Description":'Number of supermarkets within 1 mile', "Actionable":False},
    'hotel' : {"Category": "Discrete", "Description":'Number of hotels within 1 mile', "Actionable":False},
    'farmyard' : {"Category": "Discrete", "Description":'Number of farmyards within 1 mile', "Actionable":False},
    'water_tower' : {"Category": "Discrete", "Description":'Number of water towers within 1 mile', "Actionable":False},
    'christian_catholic' : {"Category": "Discrete", "Description":'Number of catholic churches within 1 mile', "Actionable":False},
    'jewish' : {"Category": "Discrete", "Description":'Number of synagogues within 1 mile', "Actionable":False},
    'muslim' : {"Category": "Discrete", "Description":'Number of mosques within 1 mile', "Actionable":False},
    'garden_centre' : {"Category": "Discrete", "Description":'Number of garden centers within 1 mile', "Actionable":False},
    'christian_lutheran' : {"Category": "Discrete", "Description":'Number of lutheran churches within 1 mile', "Actionable":False}
}

In [6]:
housing_coords["GarageCars"].value_counts()

2.0    1392
1.0     674
3.0     277
0.0     113
4.0      14
5.0       1
Name: GarageCars, dtype: int64